# Sentiment Agent 情緒分析測試

本測試展示：**市場情緒分析 Agent 的完整工作流程**

使用 5 個自訂情緒分析工具（@function_tool）進行手動測試：
- calculate_fear_greed_index
- analyze_money_flow
- analyze_news_sentiment
- analyze_social_sentiment
- generate_sentiment_signals

## Step 1: 環境設置

In [ ]:
import sys
from pathlib import Path
from dotenv import load_dotenv

from agents import (
    Runner,
    gen_trace_id,
    trace,
    WebSearchTool,
    CodeInterpreterTool,
)
from agents.mcp import MCPServerStdio

load_dotenv()

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'backend' / 'src'))

from common.logger import logger

print('✓ 環境設置完成')

## Step 2: 導入必要的模組

In [ ]:
from trading.tools.sentiment_agent import (
    get_sentiment_agent,
)

print('✓ 模組導入完成')

## Step 3: 初始化 OpenAI 內建工具

In [ ]:
# WebSearchTool: 提供網路搜尋功能
web_search_tool = WebSearchTool(
    user_location=None,  # 可選：用戶位置，用於本地化搜尋結果
    filters=None,  # 可選：搜尋過濾器
    search_context_size="medium",  # 搜尋上下文大小：'low'、'medium'、'high'
)

# CodeInterpreterTool: 提供程式碼執行功能
# 必須指定 type 和 container 設置，container.type 必須為 "auto"
code_interpreter_tool = CodeInterpreterTool(
    tool_config={
        "type": "code_interpreter",
        "container": {
            "type": "auto"  # OpenAI 自動選擇最適合的容器
        },
    }
)

openai_tools = [web_search_tool, code_interpreter_tool]

## Step 4: 初始化 MCP Servers

In [ ]:
# 全局變量來管理 MCP servers
casual_market_mcp = None
mcp_memory = None
mcp_servers = []

async def setup_mcp_servers():
    """設置 MCP servers"""
    global casual_market_mcp, mcp_memory, mcp_servers
    
    # 清理舊的連接
    if casual_market_mcp:
        try:
            await casual_market_mcp.__aexit__(None, None, None)
        except:
            pass
    
    if mcp_memory:
        try:
            await mcp_memory.__aexit__(None, None, None)
        except:
            pass
    
    # 初始化 casual-market-mcp
    casual_market_mcp = MCPServerStdio(
        params={
            "command": "uvx",
            "args": [
                "--from",
                "/Users/sacahan/Documents/workspace/CasualMarket",
                "casual-market-mcp",
            ],
        }, 
        name="casual_market_mcp", 
        client_session_timeout_seconds=120  # 增加超時時間
    )
    
    # 初始化 mcp-memory-libsql
    mcp_memory = MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "mcp-memory-libsql"],
            "env": {"LIBSQL_URL": "file:./labs/sentiment_agent.db"},
        },
        name="mcp_memory",
        client_session_timeout_seconds=120,  # 增加超時時間
    )
    
    # 連接 servers
    await casual_market_mcp.__aenter__()
    await mcp_memory.__aenter__()
    
    mcp_servers = [casual_market_mcp, mcp_memory]
    
    print('✓ MCP Servers 初始化完成')
    print(f'  - casual_market_mcp: {casual_market_mcp.name}')
    print(f'  - mcp_memory: {mcp_memory.name}')
    
    return mcp_servers

async def check_mcp_connections():
    """檢查 MCP 連接狀態"""
    global casual_market_mcp, mcp_memory
    
    print("檢查 MCP 連接狀態...")
    
    try:
        if casual_market_mcp and hasattr(casual_market_mcp, '_session'):
            print(f"  - casual_market_mcp: {'已連接' if casual_market_mcp._session else '未連接'}")
        else:
            print("  - casual_market_mcp: 未初始化")
            
        if mcp_memory and hasattr(mcp_memory, '_session'):
            print(f"  - mcp_memory: {'已連接' if mcp_memory._session else '未連接'}")
        else:
            print("  - mcp_memory: 未初始化")
    except Exception as e:
        print(f"  檢查連接時發生錯誤: {e}")

# 檢查現有連接
await check_mcp_connections()

# 重新設置 MCP servers
mcp_servers = await setup_mcp_servers()

## Step 5: 建立 Sentiment Agent

In [ ]:
async def setup_sentiment_agent():
    """初始化情緒分析 Agent"""
    agent = await get_sentiment_agent(
        model_name="gpt-4o-mini",
        mcp_servers=mcp_servers,
        openai_tools=openai_tools,
        max_turns=30,
    )
    return agent

sentiment_agent = await setup_sentiment_agent()
print('✓ Sentiment Agent 建立成功')
print(f'  名稱: {sentiment_agent.name}')
print(f'  模型: {sentiment_agent.model}')
print(f'  工具數: {len(sentiment_agent.tools)}')

## 測試 1: 完整 Agent 工作流程 - 基礎情緒分析

In [ ]:
async def test_basic_analysis():
    print('\n' + '='*70)
    print("測試 1: 完整 Agent 工作流程 - 基礎情緒分析")
    print('='*70)
    
    prompt = '''請分析以下市場狀況並生成交易建議：
    
市場數據：
- 價格動能分數: 68（接近高點）
- 市場寬度: 62（大多數股票上漲）
- 波動率: 32（偏低，市場平穩）
- 賣權買權比: 40（看漲情緒）
- 大單買進: 500M 元
- 大單賣出: 350M 元
- 外資淨買: 150M 元
- 法人淨買: 100M 元

請使用情緒分析工具分析這些數據，並生成交易訊號。'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="sentiment_agent_lab", trace_id=trace_id):
            result = await Runner.run(sentiment_agent, prompt, max_turns=10)
        
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f'執行錯誤: {e}')
        print(f'錯誤類型: {type(e).__name__}')
        return None

result_1 = await test_basic_analysis()

## 測試 2: 完整 Agent 工作流程 - 極端情緒市場

In [ ]:
async def test_extreme_market():
    print('\n' + '='*70)
    print('測試 2: 完整 Agent 工作流程 - 極端情緒市場')
    print('='*70)
    
    prompt = '''分析這個極度恐慌的市場情境：
    
市場環境：
- 美聯儲突然宣佈升息
- 科技股跌幅超過 5%
- 波動率指數 VIX 上升到 45
- 投資人大量拋售
- 融資餘額急速下降
- 融券增加明顯
- 外資淨賣超 200M

請分析：
1. 目前市場情緒等級
2. 是否是買進機會
3. 建議的交易策略
4. 風險評估'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="sentiment_agent_lab", trace_id=trace_id):
            result = await Runner.run(sentiment_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_2 = await test_extreme_market()

## 測試 3: 完整 Agent 工作流程 - 綜合情緒分析

In [ ]:
async def test_comprehensive_analysis():
    print('\n' + '='*70)
    print('測試 3: 完整 Agent 工作流程 - 綜合情緒分析')
    print('='*70)
    
    prompt = '''請進行完整的市場情緒綜合分析。
    
市場情境：
- 台股最近連漲三天
- 電子股領漲，漲幅超過 3%
- 成交量溫和增加
- 融資餘額小幅下降
- 外資持續買超 100M
- 法人買超 80M
- 大單主要集中在電子和金融
- 新聞面偏正面，有多篇 AI 相關利多報導
- 社群媒體上台積電、聯發科相關討論熱度很高

請使用所有可用的情緒分析工具進行多維度分析，最後生成明確的交易訊號和策略建議。'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行綜合分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="sentiment_agent_lab", trace_id=trace_id):
            result = await Runner.run(sentiment_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None
    
    return result

result_3 = await test_comprehensive_analysis()

## 清理資源

In [ ]:
async def cleanup_mcp_servers():
    """清理 MCP servers 連接"""
    global casual_market_mcp, mcp_memory
    
    try:
        if casual_market_mcp:
            await casual_market_mcp.__aexit__(None, None, None)
            print('✓ casual_market_mcp 已關閉')
        
        if mcp_memory:
            await mcp_memory.__aexit__(None, None, None)
            print('✓ mcp_memory 已關閉')
            
        print('✓ 所有 MCP servers 已清理完成')
        
    except Exception as e:
        print(f'清理過程中發生錯誤: {e}')

# 測試完成後執行清理
await cleanup_mcp_servers()